In [1]:
import sys
sys.path.append('../')
from src.geometry import Rectangle1D, cartesian_product
import numpy as np
%load_ext autoreload
%autoreload 2
import torch

In [2]:
# def sampler():
#     domain_x = Rectangle1D(min=-1, max=1)
#     domain_t = Rectangle1D(min=0, max=1)

#     while True:
#         np_dict = {
#             "interior": cartesian_product(
#                 domain_t.sample_interior(num_samples=10),
#                 domain_x.sample_interior(num_samples=100),
#             ),
#             "boundary": cartesian_product(
#                 domain_t.sample_interior(num_samples=10),
#                 domain_x.sample_boundary(num_samples=1),
#             ),
#             "initial": cartesian_product(
#                 np.array([0]), domain_x.sample_interior(num_samples=100)
#             ),
#         }
#         np_dict = {k: np.reshape(v, (-1, 2)) for k, v in np_dict.items()}
#         torch_dict = {
#             k: torch.tensor(v, requires_grad=True) for k, v in np_dict.items()
#         }

#         yield torch_dict

In [4]:
from torch.utils.data import IterableDataset

class DataGenerator1D(IterableDataset):
    def __init__(self, domain_x, domain_t):
        self.domain_x = domain_x
        self.domain_t = domain_t

    def __iter__(self):
        return self

    def __next__(self):
        np_dict = {
            "interior": cartesian_product(
                self.domain_t.sample_interior(num_samples=10),
                self.domain_x.sample_interior(num_samples=100),
            ),
            "boundary": cartesian_product(
                self.domain_t.sample_interior(num_samples=10),
                self.domain_x.sample_boundary(num_samples=1),
            ),
            "initial": cartesian_product(
                np.array([0]), self.domain_x.sample_interior(num_samples=100)
            ),
        }
        np_dict = {k: np.reshape(v, (-1, 2)) for k, v in np_dict.items()}
        torch_dict = {
            k: torch.tensor(v, requires_grad=True) for k, v in np_dict.items()
        }
        return torch_dict

In [5]:
sampler = DataGenerator1D(domain_x=Rectangle1D(min=-1, max=1), domain_t=Rectangle1D(min=0, max=1))

In [6]:
torch_dict = next(sampler)

In [7]:
for k, v in torch_dict.items():
    print(k, v.shape)

interior torch.Size([1000, 2])
boundary torch.Size([10, 2])
initial torch.Size([100, 2])


In [17]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    sampler,
    batch_size = None,
    pin_memory=True,
    num_workers=1,
    )

In [18]:
batch = next(iter(dataloader))

In [19]:
for k, v in batch.items():
    print(k, v.shape)

interior torch.Size([1000, 2])
boundary torch.Size([10, 2])
initial torch.Size([100, 2])
